In [4]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests

rawpath = '../raw'

In [45]:
df = pd.read_excel(f"{rawpath}/Aussteller.xlsx",names=['name','halle','stand'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df

,name,halle,stand
0,13 edition Alexandra Frohloff,Halle 2,G534
1,1a-faksimile,Halle 2,E403
2,3. Oktober - Deutschland singt und klingt e.V.,Halle 4,B322
3,360 GRAD Verlag,Halle 3,B308
4,3L Verlag,Halle 2,K107
...,...,...,...
1416,Zeitverlag Gerd Bucerius,Halle 2,F103
1417,Zirkular,Halle 4,E209
1418,zu Klampen Verlag GbR,Halle 5,D218
1419,Zweitausendeins,Halle 4,B304


In [46]:
# Basisurl für GND Dateien und dictionary für die Bestandteile der URL 
base_url = 'https://services.dnb.de/sru/cbs?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [47]:
def gnd_abfrage(row):
    params.update({'query': f'KOR="{row.name}" AND BBG=Tb*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None


            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [48]:
df['idn'] = df.apply(gnd_abfrage, axis=1)
df

,name,halle,stand,idn
0,13 edition Alexandra Frohloff,Halle 2,G534,"[{'idn': '998276634'}, {'idn': '998268941'}, {..."
1,1a-faksimile,Halle 2,E403,"[{'idn': '1258828480'}, {'idn': '970358962'}, ..."
2,3. Oktober - Deutschland singt und klingt e.V.,Halle 4,B322,"[{'idn': '103629370X'}, {'idn': '1097620611'},..."
3,360 GRAD Verlag,Halle 3,B308,"[{'idn': '1097620611'}, {'idn': '1037801997'},..."
4,3L Verlag,Halle 2,K107,"[{'idn': '320470563'}, {'idn': '992031486'}, {..."
...,...,...,...,...
1416,Zeitverlag Gerd Bucerius,Halle 2,F103,"[{'idn': '972974792'}, {'idn': '975722603'}, {..."
1417,Zirkular,Halle 4,E209,"[{'idn': '99469721X'}, {'idn': '950963429'}, {..."
1418,zu Klampen Verlag GbR,Halle 5,D218,"[{'idn': '1121925421'}, {'idn': '1168655366'},..."
1419,Zweitausendeins,Halle 4,B304,"[{'idn': '965120406'}, {'idn': '055996302'}]"


In [39]:
df.to_excel('aussteller1.xlsx')